## CIMIS ETo Exploratory Data Analysis
### Arman Ahmadi
https://doi.org/10.1016/j.scitotenv.2022.157823

### Setup

#### Importing Modules and Libraries

In [ ]:
# --- Notebook setup: paths, imports, seeds ---
from pathlib import Path
import random
import numpy as np
import pandas as pd
import sklearn as skl
import seaborn as sns
import glob

# Paths (repo-relative)
ROOT = Path(__file__).resolve().parents[1] if\
'__file__' in globals() else\
      Path.cwd().resolve().parent
DATA_DIR = ROOT / 'data' / 'sample'
OUT_DIR = ROOT / 'outputs'
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Core plotting
import matplotlib.pyplot as plt

# Metrics
from sklearn.metrics import mean_absolute_error, r2_score

# Remove warnings
import warnings
warnings.filterwarnings("ignore")


### Data Importing and Preprocessing

#### Import Data

In [ ]:
path = DATA_DIR
all_files = glob.glob(str(path / '*.csv'))

stations = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    stations.append(df)
    

#### Data Preprocessing

In [ ]:
# Changing the columns' names

for i in range(len(stations)):
  stations[i].columns = ['StnID', 'StnName', 'EToZone', 'Elevation', 'CIMISRegion', 'Date', 'Jul', 'ETo', 'qc0', \
                    'Precipitation', 'qc1', 'SolRad', 'qc2', 'NetRad', 'qc3', 'MaxVapPres', 'qc4', \
                    'MinVapPres', 'qc5', 'AvgVapPres', 'qc6', 'MaxAirTemp', 'qc7', 'MinAirTemp', 'qc8', \
                    'AvgAirTemp', 'qc9', 'MaxRelHum', 'qc10', 'MinRelHum', 'qc11', 'AvgRelHum', 'qc12', \
                    'DewPoint', 'qc13', 'AvgWindSpeed', 'qc14', 'RoseNNE', 'qc15', 'RoseENE', 'qc16', \
                    'RoseESE', 'qc17', 'RoseSSE', 'qc18', 'RoseSSW', 'qc19', 'RoseWSW', 'qc20', \
                    'RoseWNW', 'qc21', 'RoseNNW', 'qc22', 'WindRun', 'qc23', 'MaxSoilTemp', 'qc24', \
                    'MinSoilTemp', 'qc25', 'AvgSoilTemp', 'qc26', 'PMETo', 'qc27', 'PMETr', 'qc28', \
                    'BattVolt', 'qc29', 'Exp1', 'Exp2']


# Removing rows containing NaN in essential columns (where there is
#   no reading for essential variables)

StationsNan = []
for i in range(len(stations)):
  StationsNan.append(stations[i].dropna(subset=['Elevation', 'MaxAirTemp', 'MinAirTemp', 'AvgVapPres',\
                                                 'NetRad', 'AvgWindSpeed', 'Precipitation']))


# Merging all data and calculating the number and proportion of NaNs

AllStn = pd.concat(stations, axis=0, ignore_index=True)
BeforeNaNDrop = len(AllStn)
AllStnNan = pd.DataFrame.reset_index(AllStn.dropna(subset=['Elevation', 'MaxAirTemp', 'MinAirTemp', 'AvgVapPres',\
                                                 'NetRad', 'AvgWindSpeed', 'Precipitation']))

AfterNaNDrop = len(AllStnNan)
print('Number of samples BEFORE removing rows containing NaN: ', BeforeNaNDrop)
print('Number of samples AFTER removing rows containing NaN: ', AfterNaNDrop)
print('Number of removed samples due to NaN: ', BeforeNaNDrop-AfterNaNDrop)
print('Proportion of removed samples due to NaN: ', (BeforeNaNDrop-AfterNaNDrop)/BeforeNaNDrop)
print()


# Removing rows with problematic data flags ('R') and calculating the number and proportion of QC flags

StationsNanQc = []
for i in range(len(StationsNan)):
  A1 = StationsNan[i][~StationsNan[i]['qc2'].str.contains('R')]
  A2 = A1[~A1['qc3'].str.contains('R')]
  A3 = A2[~A2['qc4'].str.contains('R')]
  A4 = A3[~A3['qc5'].str.contains('R')]
  A5 = A4[~A4['qc6'].str.contains('R')]
  A6 = A5[~A5['qc7'].str.contains('R')]
  A7 = A6[~A6['qc8'].str.contains('R')]
  A8 = A7[~A7['qc9'].str.contains('R')]
  A9 = A8[~A8['qc10'].str.contains('R')]
  A10 = A9[~A9['qc11'].str.contains('R')]
  A11 = A10[~A10['qc12'].str.contains('R')]
  A12 = A11[~A11['qc13'].str.contains('R')]
  A13 = A12[~A12['qc14'].str.contains('R')]
  A14 = A13[~A13['qc27'].str.contains('R')]
  A15 = A14[~A14['qc1'].str.contains('R')]

  StationsNanQc.append(A15)

AllStnNanQc = pd.concat(StationsNanQc, axis=0, ignore_index=True)

AfterQcDrop = len(AllStnNanQc)
print('Number of samples AFTER removing rows containing QC flags: ', AfterQcDrop)
print('Number of removed samples due to QC flags: ', AfterNaNDrop-AfterQcDrop)
print('Proportion of removed samples due to QC flags: ', (AfterNaNDrop-AfterQcDrop)/BeforeNaNDrop)


# Selecting a subset of important variables

columns = [0,2,3,5,6,9,11,13,15,17,19,21,23,25,27,29,31,33,35,55,57,59,61]
StationsNanQcSelected = []
for i in range(len(StationsNanQc)):
  StationsNanQcSelected.append(StationsNanQc[i].iloc[:, columns])

AllStnNanQcSelected = pd.concat(StationsNanQcSelected, axis=0, ignore_index=True)


# Renaming the final subsets to use in the following sections

stn = StationsNanQcSelected
AllStn = AllStnNanQcSelected

# Rounding ETo zones and StnID to an integer number

AllStn.EToZone = AllStn.EToZone.astype(int)
AllStn.StnID = AllStn.StnID.astype(int)


In [ ]:
# Adding more meteorological variables and Calculating Daily ETo

# Saturation vapor pressure (kPa) at the maximum and minimum daily air temperature (deg.C)
AllStn['es_max'] = 0.6108 * np.exp((17.27*AllStn.MaxAirTemp) / (AllStn.MaxAirTemp+237.3))
AllStn['es_min'] = 0.6108 * np.exp((17.27*AllStn.MinAirTemp) / (AllStn.MinAirTemp+237.3))


# Mean daily saturation vapor pressure (kPa)
AllStn['SatVapPres'] = (AllStn.es_max + AllStn.es_min) / 2


# Vapor pressure deficit (kPa)
AllStn['VPD'] = AllStn.SatVapPres - AllStn.AvgVapPres


# Beta: barometric pressure in kPa as a function of elevation in meters
AllStn['Beta'] = 101.3 * (((293-0.0065*AllStn.Elevation)/293) ** 5.26)

# Gamma: psychrometric constant in kPa/deg.C
AllStn['Gamma'] = 0.00163 * (AllStn.Beta / 2.45)

# e0: saturation vapor pressure at mean daily temperature
AllStn['Tmean'] = (AllStn.MaxAirTemp + AllStn.MinAirTemp) / 2
AllStn['e0'] = 0.6108 * np.exp((17.27*AllStn.Tmean) / (AllStn.Tmean+237.3))

# Delta: slope of the saturation vapor pressure curve (kPa/deg.C) at mean air temperature (Tmean)
AllStn['Delta'] = (4099*AllStn.e0) / ((AllStn.Tmean+237.3)**2)

# Ro: radiation term of the PM equation for short canopy reference ET
AllStn['Ro'] = (0.408*AllStn.Delta*(AllStn.NetRad*0.0864)) / (AllStn.Delta + \
                      AllStn.Gamma*(1+0.34*AllStn.AvgWindSpeed))

# Ao: aerodynamic term of the PM equation for short canopy reference ET
AllStn['Ao'] = (((900*AllStn.Gamma)/(AllStn.Tmean+273))*AllStn.AvgWindSpeed*(AllStn.SatVapPres-AllStn.AvgVapPres))/\
  (AllStn.Delta+AllStn.Gamma*(1+0.34*AllStn.AvgWindSpeed))

AllStn['Daily_PMETo'] = AllStn.Ro + AllStn.Ao


In [ ]:
# Comaring Daily PM ETo with summation of hourly PM ETo

sns.set_theme(font_scale=1.4,style='white')

AllStn['Daily-HourlySum'] = AllStn.Daily_PMETo - AllStn.PMETo

plt.figure(figsize=(10,10))

sns.displot(AllStn[['Daily-HourlySum']])

plt.tight_layout()
plt.xlabel('Difference in ETo (mm)')
plt.xlim([-2,2])

# plt.savefig('Daily-HourlySum.jpg', dpi=600)

### Data Exploration

#### Initial Exploration & Visualization

In [ ]:
# Number of data points in each ETo zone

AllStn.groupby(['EToZone']).size()

In [ ]:
# Box plots (distributions)

AllStnImpVar = AllStn[['EToZone','SolRad','AvgAirTemp','DewPoint','AvgWindSpeed','PMETo']]

BoxPlotData = AllStnImpVar
BoxPlotData.columns = ['ETo Zone','Sol Rad (W/m^2)','Avg Air Temp (deg.C)',\
                       'Avg Vap Pres (kPa)','Avg Wind Speed (m/s)','PM ETo (mm/day)']
f, axes = plt.subplots(5,1, figsize=(6,18))

PROPS = {
    'boxprops':{'facecolor':'none', 'edgecolor':'blue'},
    'medianprops':{'color':'red'},
    'whiskerprops':{'color':'cornflowerblue'},
    'capprops':{'color':'cornflowerblue'}
}

y_vars = [ 'PM ETo (mm/day)', 'Sol Rad (W/m^2)', 'Avg Air Temp (deg.C)', 'Avg Vap Pres (kPa)',
    'Avg Wind Speed (m/s)']

for i, y in enumerate(y_vars):
    sns.boxplot(
        data=AllStnImpVar,
        x='ETo Zone',
        y=y,
        showfliers=False,
        linewidth=1.3,
        ax=axes[i],
        palette='tab20'
    )


for i in range(4):
    axes[i].get_xaxis().set_visible(False)

sns.set(font_scale=1.6,style='whitegrid')
plt.tight_layout()

# plt.savefig('boxplots.jpg', dpi=600)


In [ ]:
# Converting the dates to an acceptable format for Python

AllStn['Date']= pd.to_datetime(AllStn['Date'])

In [ ]:
# Subset data based on seasons

AllStnSpring = AllStn[(79<=AllStn.Jul) & (AllStn.Jul < 172)]
AllStnSummer = AllStn[(172<=AllStn.Jul) & (AllStn.Jul < 265)]
AllStnFall = AllStn[(265<=AllStn.Jul) & (AllStn.Jul < 355)]
AllStnWinter = AllStn[(355<=AllStn.Jul) | (AllStn.Jul < 79)]


# Adding season as a new column

AllStnSpring['Season'] = 'Spring'
AllStnSummer['Season'] = 'Summer'
AllStnFall['Season'] = 'Fall'
AllStnWinter['Season'] = 'Winter'

AllStn = pd.concat([AllStnSpring,AllStnSummer,AllStnFall,AllStnWinter],\
                   axis=0, ignore_index=True)

# Adding month as a new column

AllStn['Month'] = pd.DatetimeIndex(AllStn['Date']).month

# Adding year as a new column

AllStn['Year'] = pd.DatetimeIndex(AllStn['Date']).year


# Number of samples per each season

lenSpring = len(AllStnSpring)
lenSummer = len(AllStnSummer)
lenFall = len(AllStnFall)
lenWinter = len(AllStnWinter)
print('Number of samples in spring: ', lenSpring)
print('Number of samples in summer: ', lenSummer)
print('Number of samples in fall: ', lenFall)
print('Number of samples in winter: ', lenWinter)

#### Pearson Correlation

In [ ]:
# Pearson correlation heatmaps for each season

sns.set_theme(font_scale=1.1,style='white')

PCHData = AllStn[[  'PMETo', 'SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres',\
                                     'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                     'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                     'MinSoilTemp', 'AvgSoilTemp', 'Season']]
PCHData.columns = ['PM ETo', 'Sol Rad', 'Max Vap Pres', 'Min Vap Pres',\
                                    'Avg Vap Pres', 'Max Rel Hum', 'Min Rel Hum', 'Avg Rel Hum', 'Dew point',\
                                    'Avg Wind Speed','Max Air Temp', 'Min Air Temp', 'Avg Air Temp',\
                                    'Max Soil Temp', 'Min Soil Temp', 'Avg Soil Temp', 'Season']

f, axes = plt.subplots(1,4, figsize=(10,8))

corr_vars = ['PM ETo', 'Sol Rad', 'Max Vap Pres', 'Min Vap Pres', 'Avg Vap Pres',
    'Max Rel Hum', 'Min Rel Hum', 'Avg Rel Hum', 'Dew point',
    'Avg Wind Speed', 'Max Air Temp', 'Min Air Temp', 'Avg Air Temp',
    'Max Soil Temp', 'Min Soil Temp', 'Avg Soil Temp']

seasons = ['Spring', 'Summer', 'Fall', 'Winter']

for i, season in enumerate(seasons):
    corr_matrix = (
        PCHData[PCHData.Season == season][corr_vars]
        .corr()[['PM ETo']]
        .sort_values(by='PM ETo', ascending=False)
    )
    sns.heatmap(
        corr_matrix,
        vmin=-1, vmax=1, annot=True,
        cmap='RdBu_r', cbar=False,
        ax=axes[i], xticklabels=False
    )
    axes[i].set_title(season)

plt.tight_layout()

# plt.savefig('correlation_seasons.jpg', dpi=600)

In [ ]:
# Pearson correlation heatmap for California

PCHData1 = AllStn[[ 'PMETo', 'SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres', 'VPD',\
                                     'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                     'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                     'MinSoilTemp', 'AvgSoilTemp', 'Season', 'Month','EToZone']]

meteo_var = ['PMETo', 'SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres','VPD',\
                                    'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                    'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                    'MinSoilTemp', 'AvgSoilTemp']

PC_total = [ [ None ] * 17 ] * 22

PC_total[0] = np.array(PCHData1[meteo_var].corr()[['PMETo']]).transpose()

zones = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18]

for i, zone in enumerate(zones, start=1):
    PC_total[i] = (
        np.array(
            PCHData1[PCHData1.EToZone == zone][meteo_var]
            .corr()[['PMETo']]
        ).transpose()
    )

PC_total[18] = np.array(PCHData1[PCHData1.Season=='Spring'][meteo_var].corr()[['PMETo']]).transpose()
PC_total[19] = np.array(PCHData1[PCHData1.Season=='Summer'][meteo_var].corr()[['PMETo']]).transpose()
PC_total[20] = np.array(PCHData1[PCHData1.Season=='Fall'][meteo_var].corr()[['PMETo']]).transpose()
PC_total[21] = np.array(PCHData1[PCHData1.Season=='Winter'][meteo_var].corr()[['PMETo']]).transpose()

PC_total =np.array(PC_total).reshape((22, 17))


In [ ]:
# Visualizing the heatmap

plt.figure(figsize=(16,16))
ylabels = ['California','Zone 1','Zone 2','Zone 3','Zone 4','Zone 5','Zone 6',\
           'Zone 7','Zone 8','Zone 9','Zone 10','Zone 12','Zone 13',\
           'Zone 14','Zone 15','Zone 16','Zone 17','Zone 18',\
           'Spring','Summer','Fall','Winter']
xlabels = ['PM ETo','Sol Rad', 'Max Vap Pres', 'Min Vap Pres', 'Avg Vap Pres','VPD',\
           'Max Rel Hum','Min Rel Hum','Avg Rel Hum', 'Dew Point', 'Avg Wind Speed',\
           'Max Air Temp', 'Min Air Temp', 'Avg Air Temp', 'Max Soil Temp',\
           'Min Soil Temp', 'Avg Soil Temp']

z1 = sns.heatmap(PC_total, annot=True, yticklabels=ylabels, xticklabels=xlabels,\
                 cmap='RdBu_r', fmt='.2f')
plt.xticks(rotation='vertical')

sns.set_theme(font_scale=1.4,style='white')
plt.tight_layout()

# plt.savefig('Pearson_correlation.jpg', dpi=600)

In [ ]:
# Monthly Pearson correlation

PCHData1 = AllStn[[ 'PMETo', 'SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres','VPD',\
                                     'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                     'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                     'MinSoilTemp', 'AvgSoilTemp', 'Season', 'Month','EToZone']]

meteo_var = ['PMETo', 'SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres','VPD',\
                                    'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                    'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                    'MinSoilTemp', 'AvgSoilTemp']

PC_monthly = [ [ None ] * 17 ] * 12

for month in range(1, 13):
    PC_monthly[month - 1] = (
        np.array(
            PCHData1[PCHData1.Month == month][meteo_var]
            .corr()[['PMETo']]
        ).transpose()
    )

PC_monthly =np.array(PC_monthly).reshape((12, 17))


In [ ]:
# Visualizing the heatmap

plt.figure(figsize=(16,10))
ylabels = ['January', 'February', 'March', 'April', 'May', 'June', 'July',\
           'August', 'September', 'October', 'November', 'December']
xlabels = ['PM ETo','Sol Rad', 'Max Vap Pres', 'Min Vap Pres', 'Avg Vap Pres','VPD',\
           'Max Rel Hum','Min Rel Hum','Avg Rel Hum', 'Dew Point', 'Avg Wind Speed',\
           'Max Air Temp', 'Min Air Temp', 'Avg Air Temp', 'Max Soil Temp',\
           'Min Soil Temp', 'Avg Soil Temp']

z1 = sns.heatmap(PC_monthly, annot=True, yticklabels=ylabels, xticklabels=xlabels,\
                 cmap='RdBu_r', fmt='.2f')
plt.xticks(rotation='vertical')

sns.set_theme(font_scale=1.4,style='white')
plt.tight_layout()

# plt.savefig('monthly_Pearson_correlation.jpg', dpi=600)

#### Mutual Information

In [ ]:
# Mutual information for each month

meteo_var = ['SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres','VPD',\
                                    'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                    'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                    'MinSoilTemp', 'AvgSoilTemp']

data = AllStn.dropna(subset= meteo_var + ['PMETo'])

MI_months = [ [ None ] * 16 ] * 12

for month in range(1, 13):
    X = data[data.Month == month][meteo_var]
    y = data[data.Month == month][['PMETo']]
    MI_state = skl.feature_selection.mutual_info_regression(X, y)
    MI_months[month - 1] = MI_state


In [ ]:
# Visualizing monthly mutual information


plt.figure(figsize=(15,10))
ylabels = ['January', 'February', 'March', 'April', 'May', 'June', 'July',\
           'August', 'September', 'October', 'November', 'December']
xlabels = ['Sol Rad', 'Max Vap Pres', 'Min Vap Pres', 'Avg Vap Pres','VPD',\
           'Max Rel Hum','Min Rel Hum','Avg Rel Hum', 'Dew Point', 'Avg Wind Speed',\
           'Max Air Temp', 'Min Air Temp', 'Avg Air Temp', 'Max Soil Temp',\
           'Min Soil Temp', 'Avg Soil Temp']

z1 = sns.heatmap(MI_months, annot=True, yticklabels=ylabels, xticklabels=xlabels,\
                 cmap='YlGnBu', fmt='.2f')
plt.xticks(rotation='vertical')

sns.set_theme(font_scale=1.4,style='white')
plt.tight_layout()

# plt.savefig('monthly_mutual_information.jpg', dpi=600)

#### Random Forest Feature Importance

In [ ]:
# Fit Random Forest models for each month and get the feature importance

meteo_var = ['SolRad', 'MaxVapPres', 'MinVapPres', 'AvgVapPres','VPD',\
                                    'MaxRelHum','MinRelHum','AvgRelHum', 'DewPoint', 'AvgWindSpeed',\
                                    'MaxAirTemp', 'MinAirTemp', 'AvgAirTemp', 'MaxSoilTemp',\
                                    'MinSoilTemp', 'AvgSoilTemp']

data = AllStn.dropna(subset= meteo_var + ['PMETo'])

importance_months = [ [ None ] * 16 ] * 12

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

for month in range(1, 13):
    X = data[data.Month == month][meteo_var]
    y = data[data.Month == month][['PMETo']]
    model.fit(X, y)
    importance_months[month - 1] = model.feature_importances_


In [ ]:
# Visualizing monthly random forest feature importance


plt.figure(figsize=(15,10))
ylabels = ['January', 'February', 'March', 'April', 'May', 'June', 'July',\
           'August', 'September', 'October', 'November', 'December']
xlabels = ['Sol Rad', 'Max Vap Pres', 'Min Vap Pres', 'Avg Vap Pres','VPD',\
           'Max Rel Hum','Min Rel Hum','Avg Rel Hum', 'Dew Point', 'Avg Wind Speed',\
           'Max Air Temp', 'Min Air Temp', 'Avg Air Temp', 'Max Soil Temp',\
           'Min Soil Temp', 'Avg Soil Temp']

z1 = sns.heatmap(importance_months, annot=True, yticklabels=ylabels, xticklabels=xlabels,\
                 cmap='YlGnBu', fmt='.2f')
plt.xticks(rotation='vertical')

sns.set_theme(font_scale=1.4,style='white')
plt.tight_layout()

# plt.savefig('monthly_RF_feature_importance.jpg', dpi=600)

#### Trend Analysis

In [ ]:
# Averaged Annual Data for each station that have more than 339 days of data
# for that specific year

data = AllStn[['StnID', 'EToZone', 'Year', 'Daily_PMETo', 'Precipitation',
              'SolRad', 'MaxAirTemp', 'VPD', 'AvgWindSpeed']]

annual = data.groupby(['StnID','Year'], sort=False).mean()
annual['Size'] = data.groupby(['StnID','Year'], sort=False).size()
annual = annual[annual.Size > 339]

# Zonal annual data averaged over available stations at the specific year
annual_zone = annual.groupby(['EToZone','Year'], sort=False).mean()
annual_zone = annual_zone.reset_index()

# California annual data averaged over available stations at the specific year
annual_CA = annual.groupby(['Year'], sort=False).mean()
annual_CA = annual_CA.reset_index()

# # Number of stations for each year
# annual.groupby(['Year'], sort=False).size()

In [ ]:
# Annual trends visualization

f, axes = plt.subplots(6,1, figsize=(12,24))

series = [
    ('Daily_PMETo', 365, 'ETo (mm/year)'),
    ('Precipitation', 365, 'Precipitation (mm/year)'),
    ('SolRad', 1, 'Solar Radiation (W/m^2)'),
    ('MaxAirTemp', 1, 'Max Air Temperature (°C)'),
    ('VPD', 1, 'Vapor Pressure Deficit (kPa)'),
    ('AvgWindSpeed', 1, 'Wind Speed (m/s)'),
]

x = annual_CA['Year']

for i, (col, factor, ylabel) in enumerate(series):
    y = (annual_CA[col] * factor)

    # main line
    sns.lineplot(x=x, y=y, ax=axes[i], color='blue', ls='--', marker='o', ms=8)

    # trendline
    valid = np.isfinite(x) & np.isfinite(y)
    z = np.polyfit(x[valid], y[valid], 1)   # z[0] is slope
    p = np.poly1d(z)

    # plot trendline
    sns.lineplot(x=x, y=p(x), ax=axes[i], color='red', ls='--')

    # labels & title
    axes[i].set(xlabel='Year', ylabel=ylabel, title=f'Trendline Slope = {z[0]:.3f}')

sns.set_theme(font_scale=1.5,style='whitegrid')
plt.tight_layout()
# plt.savefig('Annual_CA_Trend.jpg', dpi=600)